In [1]:
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AX4XfWjNTmEnBmZeO0f6vkYFK5COGI_0u0kY9kiUWEp1jnuAKKGzPvWxRSg

Successfully saved authorization token.


In [ ]:
# The resource I used to figure out GEE:
# https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard 

In [2]:
#importing some basic python data science modules
from datetime import datetime, date
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt

In [3]:
#Reading in data from the HotSpots Challenge (need the latitude and longitudes)
df=pd.read_csv('data/train.csv')


In [4]:
df.head()

,ID,area,date,lat,lon,burn_area,climate_aet,climate_def,climate_pdsi,climate_pet,...,landcover_1,landcover_2,landcover_3,landcover_4,landcover_5,landcover_6,landcover_7,landcover_8,population_density,precipitation
0,0_2000-04-01,0,2000-04-01,25.447,5.296,0.003688,1250.622712,0.0,-178.916305,1250.622712,...,0.0,0.350169,0.0,0.649524,0.0,0.000307,0.0,0.0,2.214262,0.198996
1,1_2000-04-01,1,2000-04-01,25.669,5.293,0.000000,1238.019166,0.0,-150.779947,1238.019166,...,0.0,0.429049,0.0,0.570644,0.0,0.000307,0.0,0.0,3.833042,0.188071
2,2_2000-04-01,2,2000-04-01,25.443,5.074,0.000000,1240.449964,0.0,-200.503858,1240.449964,...,0.0,0.238300,0.0,0.761700,0.0,0.000000,0.0,0.0,1.927303,0.211730
3,3_2000-04-01,3,2000-04-01,25.665,5.070,0.000000,1229.240077,0.0,-177.011032,1229.240077,...,0.0,0.488146,0.0,0.511854,0.0,0.000000,0.0,0.0,1.878281,0.215403
4,4_2000-04-01,4,2000-04-01,25.886,5.067,0.000307,1224.093679,0.0,-153.256111,1224.093679,...,0.0,0.322243,0.0,0.677757,0.0,0.000000,0.0,0.0,1.968818,0.199975


In [ ]:
#NOTE: I am a little concerned by this data which we got from the HotSpots challenge, because the lat and lon are switched (a simple google search shows what general lon and lats of the DRC are)
#We will keep this in mind to get the correct data from GEE later 

In [5]:
#Basic function to turn arrays into Dataframe (This function was from GEE python tutorial, I just edited it a little bit)
def ee_array_to_df(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Keep the columns of interest.
    df = df[[ *list_of_bands]]

    return df

In [6]:
#Getting an image collection from GEE for burn area
#To keep this up to date, we could replace '2021-06-01', with the present date
burn_data = ee.ImageCollection('MODIS/006/MCD64A1').filter(ee.Filter.date('2015-01-01', '2021-06-01'))
#If you want to check out this burn data API, you can search 'MODIS/006/MCD64A1' in GEE
burnarea= burn_data.select('BurnDate') # we just need the 'BurnDate' band from this image collection 

In [7]:
# This takes the original points from the HotSpots data challenge
#NOTE: Normally, we need to input [x,y] into GEE for a multipoint geometry, which translates to [lon,lat], but since lon/lat are switched in the original data, we will switch it here
points=[]
for i in range(0,len(df.area.unique())):
      points.append(list(df[['lat','lon']].loc[df.area==i].values[0]))
        
points =ee.Geometry.MultiPoint(points)

[[25.447, 5.296],
 [25.669, 5.292999999999998],
 [25.443, 5.074],
 [25.665, 5.07],
 [25.886, 5.067],
 [26.108, 5.063],
 [26.329, 5.059],
 [26.55, 5.055],
 [26.771, 5.051],
 [26.991, 5.047],
 [27.212, 5.043],
 [27.432, 5.038],
 [19.181, 4.92],
 [19.406, 4.919],
 [19.631, 4.917],
 [19.855, 4.916],
 [20.08, 4.914],
 [24.107, 4.871],
 [24.329, 4.868],
 [24.551, 4.865],
 [24.774, 4.862],
 [24.996, 4.8580000000000005],
 [25.218000000000004, 4.855],
 [25.44, 4.852],
 [25.661, 4.848],
 [25.883000000000006, 4.845],
 [26.104, 4.841],
 [26.325, 4.837],
 [26.546, 4.833],
 [26.767, 4.83],
 [26.987, 4.8260000000000005],
 [27.208, 4.822],
 [27.428, 4.8180000000000005],
 [27.648000000000003, 4.814],
 [18.955, 4.6960000000000015],
 [19.18, 4.695],
 [19.404, 4.6930000000000005],
 [19.629, 4.692],
 [19.854, 4.69],
 [20.078, 4.689],
 [20.303, 4.687],
 [22.989, 4.662],
 [23.212, 4.659],
 [23.658, 4.654],
 [23.881, 4.651],
 [24.104, 4.648],
 [24.326, 4.645],
 [24.548, 4.642],
 [24.771, 4.638999999999999],
 

In [9]:
#This creates an array of the data
point_buffer = 1000 #1000m buffer around each point
burn_array = burnarea.getRegion(points,point_buffer).getInfo() #Get regions gives us the time series data we want

# We get an array of time series data, which is exacly what we want
# We can also look at the lon/lat to see that they are correct for the DRC
burn_array

[['id', 'longitude', 'latitude', 'time', 'BurnDate'],
 ['2015_01_01', 30.87958789160855, 1.7023074634064934, 1420070400000, None],
 ['2015_02_01', 30.87958789160855, 1.7023074634064934, 1422748800000, None],
 ['2015_03_01', 30.87958789160855, 1.7023074634064934, 1425168000000, None],
 ['2015_04_01', 30.87958789160855, 1.7023074634064934, 1427846400000, None],
 ['2015_05_01', 30.87958789160855, 1.7023074634064934, 1430438400000, None],
 ['2015_06_01', 30.87958789160855, 1.7023074634064934, 1433116800000, None],
 ['2015_07_01', 30.87958789160855, 1.7023074634064934, 1435708800000, None],
 ['2015_08_01', 30.87958789160855, 1.7023074634064934, 1438387200000, None],
 ['2015_09_01', 30.87958789160855, 1.7023074634064934, 1441065600000, None],
 ['2015_10_01', 30.87958789160855, 1.7023074634064934, 1443657600000, None],
 ['2015_11_01', 30.87958789160855, 1.7023074634064934, 1446336000000, None],
 ['2015_12_01', 30.87958789160855, 1.7023074634064934, 1448928000000, None],
 ['2016_01_01', 30.879

In [10]:
#This creates the data frame (the list of names is the columns of the data frames)
data=ee_array_to_df(burn_array, ['id','longitude', 'latitude', 'BurnDate' ])

In [11]:
# We need to clean the data up a little bit
# The way it is formatted is that 'none' denotes a burn area of zero and any number 1-366 denotes a fire in that area on that day of the year
# We fix this by making it numerical, filling in the non-numerical places with zero, and then taking anything greater than zero and making it one
data['BurnDate']=pd.to_numeric(data['BurnDate'])
data=data.fillna(0)
data.BurnDate.loc[data.BurnDate>0]=1 #This could probably be done in a better way, but it works
data=data.rename(columns={'BurnDate':'burn_area'})


# Creating areas to forecast from our data

We want to reduce from around 4,000 locations into 20 locations, so we can create a reasonable amount of models. I chose 20 because this make the areas small enough to differentiate the different climates that are present throughout DRC.

In [25]:
#This shows max/min lon 
data.longitude.sort_values()

12452    12.419209
12442    12.419209
12441    12.419209
12440    12.419209
12439    12.419209
           ...    
307      31.095184
308      31.095184
309      31.095184
311      31.095184
324      31.095184
Name: longitude, Length: 290396, dtype: float64

In [33]:
# We also get max/min lat
data.latitude.sort_values()

87628   -13.398372
87683   -13.398372
87682   -13.398372
87681   -13.398372
87680   -13.398372
           ...    
1267      5.295569
1268      5.295569
1269      5.295569
1271      5.295569
1293      5.295569
Name: latitude, Length: 290396, dtype: float64

In [39]:
# Now, we can start to make a 5x5 grid around the DRC (this will result in only 20 locations due to the shape of the DRC)
# Since the DRC lon/lat are not going to change, we can do some basic math to figure out how we want the grid divided
lon=np.arange(12.4+3.74,31.1+3.74,3.75)
lon

array([16.14, 19.89, 23.64, 27.39, 31.14])

In [40]:
lat=np.arange(-13.5+3.8,5.5+3.8,3.8)
lat

array([-9.7, -5.9, -2.1,  1.7,  5.5])

This function makes the 20 different locations by taking all the points in a given grid's lon and lat and then takes the mean burn area for every single monthly timestamp. The 'if statement' is there because the most western part of the DRC is differenly shaped than the rest of the DRC, so I created a custom rectangle around this area.

In [72]:
#Create starting variables
lower_lat=-50
lower_lon=-50
groupdf=pd.DataFrame(columns=data.columns)


for x in lon:
    if x<=17:
            var=data.loc[(data.longitude<=x)&(data.longitude>lower_lon)&(data.latitude<=50)&(data.latitude>lower_lat)].groupby(['id']).mean()
            groupdf=pd.concat([groupdf,var])
             
    
    else:
        for y in lat:
                    var=data.loc[(data.longitude<=x)&(data.longitude>lower_lon)&(data.latitude<=y)&(data.latitude>lower_lat)].groupby(['id']).mean()
                    groupdf=pd.concat([groupdf,var])
                    lower_lat=y
    lower_lat=-50
    lower_lon=x


In [73]:
# Now we can see the data that we made:
#NOTE: the lon/lat are the mean lon/lat, and not that exact centers of each of the grids
groupdf.head()

,id,longitude,latitude,burn_area
2015_01_01,NaN,14.496897,-5.130336,0.0
2015_02_01,NaN,14.496897,-5.130336,0.0
2015_03_01,NaN,14.496897,-5.130336,0.0
2015_04_01,NaN,14.496897,-5.130336,0.0
2015_05_01,NaN,14.496897,-5.130336,0.0


In [74]:
groupdf.shape

(1520, 4)

In [75]:
#Since we know that we chose a time period of 76 months for our intial data, we can take the rows divided by 76 to get the number of areas we made (which happens to be 20)
#NOTE: If a different time period is chosen, this would have to be adjusted.
groupdf['area']=np.arange(groupdf.shape[0]/76).repeat(76)

In [76]:
groupdf.drop(columns='id',inplace=True)
groupdf.rename(columns={'longitude':'lon','latitude':'lat'},inplace=True)
groupdf.head()

In [84]:
#Finally, we can save our data as a csv
groupdf.to_csv('fixedburndata.csv')

# Additional visualization of data

In [55]:
#use folium to map points
import folium

In [78]:
df=groupdf

In [60]:
#basic function to sort points
def sort_location(location):
    sort =groupdf.loc[(groupdf.area==location)].sort_index()
    return sort

In [83]:
#we can check where the points are located in the DRC
check_map = folium.Map( location=[df['lat'].mean(),df.lon.mean()], width='100%',height='100%',zoom_start=5)

for i in range(0,int(df.shape[0]/76)):
    sort=sort_location(i).iloc[0]
    folium.Marker(
      location=[sort.lat,sort.lon ],
      radius=10,
        popup='Area'+str(i),
    ).add_to(check_map)
check_map